In [ ]:
%pathsetup lib/python

In [ ]:
import numpy as np
import pandas as pd
#import rmm_util_accessor
import itertools
import time
import re
import grid

In [ ]:
day = 11
#data = "125 17"
data = "20 82084 1650 3 346355 363 7975858 0"
data = data.split(" ")

In [ ]:
data

# Rules
If the stone is engraved with the number 0, it is replaced by a stone engraved with the number 1.

If the stone is engraved with a number that has an even number of digits, it is replaced by two stones. 
The left half of the digits are engraved on the new left stone, and the right half of the digits are engraved on the new right stone. 
(The new numbers don't keep extra leading zeroes: 1000 would become stones 10 and 0.)

If none of the other rules apply, the stone is replaced by a new stone; the old stone's number multiplied by 2024 is engraved on the new stone.

In [ ]:
def blink(this_data):
    answer = []
    for x in this_data:
        if x=='0':
            new_x = ['1']
        elif len(x)%2==0:
            new_x = [ str(int(x[:len(x)//2])), str(int(x[len(x)//2:])) ]
        else:
            new_x = [ str(int(x) * 2024) ]
        answer.extend(new_x)
    return answer

In [ ]:
%%time
for i in range(25):
    print(f"Working blink #{i}")
    data = blink(data)

In [ ]:
len(data)

In [ ]:
data[:20]

In [ ]:
a = 251533
exp = int(np.log10(a))
print(exp)

In [ ]:
exp/2

In [ ]:
n1 = int(a / 10**(exp/2+0.5))

In [ ]:
n2 = a - int(n1 * 10**(exp/2+0.5))

In [ ]:
n1, n2

In [ ]:
a = 234

In [ ]:
int(np.log10(a))%2

In [ ]:
def blink2(this_data):
    answer = []
    for x in this_data:
        if x==0:
            answer.append(1)
            continue
        exp = len(str(x))//2
        if exp%2:
            tmp = 10**(exp/2+0.5)
            n1 = int(x / tmp)
            answer.append(n1)
            answer.append(x - int(n1 * tmp))
        else:
            answer.append(x * 2024)
    return answer

In [ ]:
%%time
nerf= []
data = "20" # 82084 1650 3 346355 363 7975858 0"
data = data.split(" ")
for i in range(45):
    print(f"Working blink #{i}, len {len(data)}")
    data = blink(data)
    nerf.extend(data)

In [ ]:
d = pd.Series(nerf, name='data').astype(int)

In [ ]:
d.value_counts()

In [ ]:
vc= d[d>1000].value_counts()
vc[vc>1].sort_values(ascending=False).head(30)

In [ ]:
print(len(data))

In [ ]:
data = "20 82084 1650 3 346355 363 7975858 0"
data = data.split(" ")
data = pd.DataFrame(data, columns=["old"]) #.astype(int)

In [ ]:
data['even_length'] = data['old'].str.len()%2==0

In [ ]:
data['midpoint'] = (data['old'].str.len()/2).astype(int)

In [ ]:
def splitterL(x):
    return x['old'][:x['midpoint']]
def splitterR(x):
    return x['old'][x['midpoint']:]

In [ ]:
data['ensplittedL'] = data.apply(splitterL, axis=1)
data['ensplittedR'] = data.apply(splitterR, axis=1)

data.loc[~data['even_length'], 'ensplittedL'] = -99
data.loc[~data['even_length'], 'ensplittedR'] = -99

data['ensplittedL'] = data['ensplittedL'].astype(int)
data['ensplittedR'] = data['ensplittedR'].astype(int)

data.loc[~data['even_length'], 'ensplittedL'] = np.nan
data.loc[~data['even_length'], 'ensplittedR'] = np.nan

data.loc[~data['even_length'], 'ensplitted'] = (data['old'].astype(int)*2024).astype(str)
data.loc[data['old']=='0', 'ensplitted'] = '1'

In [ ]:
data

In [ ]:
q = data[['ensplittedL','ensplittedR','ensplitted']]

In [ ]:
pd.Series(q.to_numpy().flatten()).dropna().astype(int).astype(str)

In [ ]:
def splitterL(x):
    return x['old'][:x['midpoint']]
def splitterR(x):
    return x['old'][x['midpoint']:]

In [ ]:
def blink3(ser):
    data = pd.DataFrame(ser, columns=['old'])
    data['even_length'] = data['old'].str.len()%2==0
    data['midpoint'] = (data['old'].str.len()/2).astype(int)
    data['ensplittedL'] = data.apply(splitterL, axis=1)
    data['ensplittedR'] = data.apply(splitterR, axis=1)
    
    data.loc[~data['even_length'], 'ensplittedL'] = -99
    data.loc[~data['even_length'], 'ensplittedR'] = -99
    
    data['ensplittedL'] = data['ensplittedL'].astype(int)
    data['ensplittedR'] = data['ensplittedR'].astype(int)
    
    data.loc[~data['even_length'], 'ensplittedL'] = np.nan
    data.loc[~data['even_length'], 'ensplittedR'] = np.nan
    
    data.loc[~data['even_length'], 'ensplitted'] = (data['old'].astype(int)*2024).astype(str)
    data.loc[data['old']=='0', 'ensplitted'] = '1'
    q = data[['ensplittedL','ensplittedR','ensplitted']]    
    return pd.Series(q.to_numpy().flatten()).dropna().astype(int).astype(str)

In [ ]:
%%time
nerf = "20 82084 1650 3 346355 363 7975858 0"
nerf = nerf.split(" ")
nerf = pd.DataFrame(nerf, columns=["old"])
for i in range(15):
    print(f"Working blink #{i}")
    nerf = blink3(nerf)

In [ ]:
nerf.head()

In [ ]:
nerf.str.split('').str.len()